needed to reload modules, otherwise you will have nan errors

In [1]:
#%load_ext autoreload
#%autoreload 2

### Data

In [2]:
from data_process_and_get_mask import create_mask
from get_data import pbmc_train, pbmc_val

tf_list = ['IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7',
       'IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A',
       'STAT5B','STAT6','GTF2I','RUNX1', 'MYOD1', 'SOX2','KLF4', 'THAP11','ADNP2', 'AEBP1','AHRR', 'ALX4','APBB1','APEX1', 'ARHGAP35','ARID1A']

#['IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7', 'IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A','STAT5B', 'STAT6']
#["GTF2I","GTF3A","NRF1","ELF1","STAT1","STAT2","IRF9","STAT3","STAT4","STAT5A","STAT5B","IRF3","IRF7","IRF1","IRF5","IRF8"]

mask, train, valid = create_mask(pbmc_train, pbmc_val, tf_list)
print("Printing mask:")
print(mask)
print("\nPrinting train:")
print(train)
print("\nPrinting valid:")
print(valid)

Printing mask:
shape: (471, 32)
┌────────┬────────┬────────┬───────┬───┬────────┬───────┬───────┬───────────────┐
│ target ┆ STAT5A ┆ STAT5B ┆ STAT4 ┆ … ┆ ARID1A ┆ ADNP2 ┆ APBB1 ┆ unannotated_1 │
│ ---    ┆ ---    ┆ ---    ┆ ---   ┆   ┆ ---    ┆ ---   ┆ ---   ┆ ---           │
│ str    ┆ f64    ┆ f64    ┆ f64   ┆   ┆ f64    ┆ f64   ┆ f64   ┆ i32           │
╞════════╪════════╪════════╪═══════╪═══╪════════╪═══════╪═══════╪═══════════════╡
│ IL2    ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ IRF1   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ CCND1  ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ PRF1   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ IFNG   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ …      ┆ …      ┆ …      ┆ …     ┆ … ┆ …      ┆ …     ┆ …     ┆ …             │
│ ZNF181 ┆ 0.0    ┆ 0.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 

In [3]:
# model training
#vega, vega_losses, vega_klds, vega_mses = trainVEGA(vega, train, epochs = 20, beta = 0.0001) #takes about 2 mins on GPU # epoch 100

now use the common training functions

In [4]:
mask

target,STAT5A,STAT5B,STAT4,IRF1,STAT3,RUNX1,GTF2I,IRF2,STAT1,IRF8,STAT6,APEX1,IRF4,MYOD1,ARHGAP35,IRF3,IRF5,ALX4,STAT2,IRF7,KLF4,IRF9,AHRR,AEBP1,SOX2,THAP11,IRF6,ARID1A,ADNP2,APBB1,unannotated_1
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
"""IL2""",1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IRF1""",1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""CCND1""",1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
"""PRF1""",1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IFNG""",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZNF181""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF230""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF302""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [8]:
from training.training import run_vega_model
run_vega_model(
    model_type="vega",
    train_data=train,
    valid_data=valid,
    mask_df=mask,
    epochs=30,
    N=10,
    cond="stimulated"
)

epoch:  0  train_loss:  113.03845   valid_loss:  93.520485
epoch:  10  train_loss:  48.92899   valid_loss:  37.891956
epoch:  20  train_loss:  44.352077   valid_loss:  35.37486
epoch:  0  train_loss:  109.34229   valid_loss:  86.87133
epoch:  10  train_loss:  47.762432   valid_loss:  37.00459
epoch:  20  train_loss:  42.899746   valid_loss:  35.05467
epoch:  0  train_loss:  115.56035   valid_loss:  96.260574
epoch:  10  train_loss:  49.728848   valid_loss:  40.65053
epoch:  20  train_loss:  44.375546   valid_loss:  36.232616
epoch:  0  train_loss:  112.56797   valid_loss:  91.58539
epoch:  10  train_loss:  48.88992   valid_loss:  39.691
epoch:  20  train_loss:  43.051384   valid_loss:  35.47816
epoch:  0  train_loss:  112.14052   valid_loss:  89.61753
epoch:  10  train_loss:  49.29163   valid_loss:  38.18098
epoch:  20  train_loss:  44.19376   valid_loss:  35.91959
epoch:  0  train_loss:  113.50017   valid_loss:  90.57658
epoch:  10  train_loss:  49.198112   valid_loss:  39.91282
epoch

(VEGA(
   (decoder): DecoderVEGA(
     (sparse_layer): Sequential(
       (0): SparseLayer()
     )
   )
   (encoder): Encoder(
     (encoder): Sequential(
       (0): Linear(in_features=471, out_features=800, bias=True)
       (1): ReLU()
       (2): Dropout(p=0.3, inplace=False)
       (3): Linear(in_features=800, out_features=800, bias=True)
       (4): ReLU()
       (5): Dropout(p=0.3, inplace=False)
     )
     (mu): Sequential(
       (0): Linear(in_features=800, out_features=31, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
     (sigma): Sequential(
       (0): Linear(in_features=800, out_features=31, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
   )
 ),
           STAT5A    STAT5B     STAT4      IRF1     STAT3     RUNX1     GTF2I  \
 IL2    -0.000010 -0.000012  0.000000  0.000000 -0.000050  0.000094  0.000000   
 IRF1    0.000601  0.002115  0.003117 -0.001094 -0.000919  0.000000  0.000000   
 CCND1  -0.000261 -0.000023  0.000000  0.000031  0.000014 